In [2]:
using Revise
using ITensors
using PastaQ
push!(LOAD_PATH,"/scratch/Codes/RandomMeas_dev.jl/src/")
using RandomMeas
using ProgressBars


In [3]:
N = 40
ξ = siteinds("Qubit", N)
B = 1.
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising ZZ interactions
  ampo .+= -1, "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field X
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ)
H2 = multiply(H,H)
# Density-matrix renormalization group
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = randomMPS(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)
# Run 
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψ = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-50.079934626842885  maxlinkdim=4 maxerr=1.47E-15 time=17.834
After sweep 2 energy=-50.50173277758765  maxlinkdim=15 maxerr=9.56E-11 time=0.073
After sweep 3 energy=-50.560094581012464  maxlinkdim=25 maxerr=9.96E-11 time=0.080
After sweep 4 energy=-50.568229174748154  maxlinkdim=26 maxerr=9.73E-11 time=0.091
After sweep 5 energy=-50.56932436017555  maxlinkdim=23 maxerr=9.68E-11 time=0.082

Ground state energy:  -50.56932436017555

---------------------------------------



In [4]:
η = 10
m = 4
# Build the gate structure
circuit = []
for d in 1:η
    xx_layer = [("Rxx", (j, j + 1), (ϕ=1*d/η/m,)) for j in 1:(N - 1)]
    z_layer = [("Rz", j, (θ=2*B/m,)) for j in 1:N]
    for t in 1:m
        append!(circuit, xx_layer)
        append!(circuit, z_layer)
    end
end
ψ0 = MPS(ξ,["Up" for n in 1:N]);

In [5]:
ψt = runcircuit(ψ0, circuit; cutoff=1E-8)
orthogonalize!(ψt,1)
ψt[1] /= norm(ψt[1])
E_th = real(inner(ψt', H,ψt))
println("final energy ",E_th)
E2_th = real(inner(ψt', H2,ψt))-E_th^2
println("final energy variance ",E2_th)

final energy -48.128696873191444
final energy variance 11.581820887635331


In [6]:
Nu = 2000
NM = 100
ρ = Vector{ITensor}(undef,N);
data = zeros(Int8,(NM,N));

In [7]:
Es = 0.
E2s = 0.
for r in ProgressBar(1:Nu, printing_delay=20)
        u = get_rotations(ξ,1)
        get_RandomMeas_MPS!(data,ψt,u,NM)
        for m in 1:NM
            get_shadow_factorized!(ρ,data[m,:],ξ,u,ones(N),N)
            Es += get_expect_shadow(H,ρ,ξ)/Nu/NM
            E2s += get_expect_shadow(H2,ρ,ξ)/Nu/NM
        end
end
E2s -= Es^2
println("estimated energy ",Es)
println("estimated energy variance ",E2s)

0.0%┣                                        ┫ 0/2.0k [00:05<-2:-48:-55, -5s/it]
1.6%┣▊                                            ┫ 31/2.0k [00:26<27:54, 1it/s]
4.7%┣██▏                                          ┫ 94/2.0k [00:46<15:34, 2it/s]
7.7%┣███▍                                        ┫ 154/2.0k [01:06<13:12, 2it/s]
11.0%┣████▊                                      ┫ 220/2.0k [01:26<11:37, 3it/s]
14.2%┣██████▏                                    ┫ 285/2.0k [01:46<10:39, 3it/s]
18.1%┣███████▉                                   ┫ 362/2.0k [02:06<09:32, 3it/s]
21.5%┣█████████▎                                 ┫ 430/2.0k [02:26<08:56, 3it/s]
24.6%┣██████████▋                                ┫ 493/2.0k [02:47<08:30, 3it/s]
28.0%┣████████████                               ┫ 559/2.0k [03:07<08:03, 3it/s]
31.0%┣█████████████▎                             ┫ 620/2.0k [03:27<07:42, 3it/s]
34.0%┣██████████████▋                            ┫ 680/2.0k [03:47<07:22, 3it/s]
37.1%┣████████████████      

estimated energy -48.251165494012895
estimated energy variance 11.167012676557533


100.0%┣█████████████████████████████████████████┫ 2.0k/2.0k [10:59<00:00, 3it/s]
100.0%┣█████████████████████████████████████████┫ 2.0k/2.0k [10:59<00:00, 3it/s]
